<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Parcial_2_SyS_2025-1/Punto1_Parcial2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Dashboard Interactivo para Sistemas de Segundo Orden

**Punto 1 Parcial 2 SyS**  
**Omar Andrés Rodriguez Quiceno**  

---

Este notebook está diseñado para estudiar y comparar dos modelos equivalentes de sistemas de segundo orden:

1. **Sistema mecánico masa–resorte–amortiguador**  
2. **Circuito eléctrico R–L–C**

A través de una aplicación interactiva desarrollada con **Streamlit**, el usuario podrá:

- Seleccionar el tipo de respuesta del sistema:
  - Subamortiguada  
  - Sobreamortiguada  
  - Critica  
  - Inestable  
- Ajustar dinámicamente:
  - El factor de amortiguamiento \(\zeta\)  
  - La frecuencia natural \(\omega_n\)  
- Visualizar en **lazo abierto** y **lazo cerrado**:
  - Diagrama de Bode  
  - Diagrama de polos y ceros  
  - Respuestas al impulso, escalón y rampa  
  - Parámetros temporales:
    - Tiempo de levantamiento  
    - Sobreimpulso máximo  
    - Tiempo de sobreimpulso  
    - Tiempo de establecimiento  
  - Valores calculados de los componentes de cada modelo:
    - Masa \(m\), resorte \(k\) y amortiguador \(c\)  
    - Inductancia \(L\), capacitancia \(C\) y resistencia \(R\)  

---




## Instalación de Dependencias y Configuración del Entorno

En este bloque configuramos todo lo necesario para ejecutar nuestra aplicación Streamlit en Google Colab:

1. Instalamos **Streamlit**, la librería principal para crear el dashboard.  
2. Verificamos la ruta y versión de Streamlit para asegurarnos de que la instalación fue correcta.  
3. Instalamos **Librosa**, necesaria si más adelante queremos procesar audio.  
4. Descargamos y configuramos **Cloudflared**, que nos permitirá exponer nuestro servidor local de Streamlit a Internet.


In [ ]:
# ── Instalación de Streamlit ───────────────────────────────────────────────
!pip install streamlit              # Instala Streamlit para crear la interfaz web
!which streamlit                    # Muestra la ruta donde se ha instalado el ejecutable
!streamlit --version                # Comprueba la versión de Streamlit instalada

# ── Instalación de Librosa ────────────────────────────────────────────────
!pip install librosa --quiet        # Instala Librosa para procesamiento de audio (modo silencioso)

# ── Descarga y configuración de Cloudflared ─────────────────────────────────
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
                                     # Descarga la última versión de cloudflared para Linux
!chmod +x cloudflared                # Asigna permisos de ejecución al binario descargado
!mv cloudflared /usr/local/bin/      # Mueve el binario a una ubicación accesible globalmente


/usr/local/bin/streamlit
Streamlit, version 1.46.1


## Creación del Directorio de Páginas

En este bloque verificamos la existencia de la carpeta `pages/`—donde almacenaremos cada una de las subpáginas de la aplicación Streamlit—y la creamos si no está presente. Esto previene errores al guardar los archivos generados para cada sección del dashboard.


In [ ]:
import os  # Módulo para operaciones con el sistema de archivos

# ── Creación del directorio 'pages' si no existe ──────────────────────────
os.makedirs("pages", exist_ok=True)  # exist_ok=True evita excepción si ya existe

print("✅ Carpeta 'pages/' lista.")  # Mensaje de confirmación para el usuario


✅ Carpeta 'pages/' lista.


## Script Principal: `0_📊_Simulacion_Sistemas.py`

En este bloque creamos el archivo que inicia la aplicación Streamlit, definiendo tanto la **configuración** de la página como la **estructura inicial** del dashboard. Aquí se establecen:

- El título y el ícono de la aplicación.  
- Un mensaje de bienvenida en la barra lateral.  
- Una descripción general de las funcionalidades principales del panel interactivo.  


In [ ]:
%%writefile 0_📊_Simulacion_Sistemas.py
import streamlit as st

# ── Configuración de la página ────────────────────────────────────────────
st.set_page_config(
    page_title="Simulación de Sistemas",
    page_icon="📊",
)

# ── Título principal y bienvenida ────────────────────────────────────────
st.write("# 📊 Sistemas de Segundo Orden")
st.sidebar.success("Elige una simulación en el menú lateral.")

# ── Introducción y descripción del dashboard ──────────────────────────────
st.markdown(
    """
    ##
    Este panel interactivo forma parte del segundo parcial de Señales y Sistemas. Su propósito es brindar una **visualización intuitiva** de la dinámica de dos modelos de segundo orden:

    ### Funcionalidades principales

    - **Elegir el tipo de comportamiento**: subamortiguado, sobreamortiguado, críticamente amortiguado o inestable.
    - **Ajustar parámetros clave**: el amortiguamiento (ζ) y la frecuencia natural (ωₙ).
    - **Comparar modelos**: ver tanto el sistema mecánico (masa–resorte–amortiguador) como su equivalente eléctrico (R–L–C).
    - **Visualizar gráficos clave**:
        - Diagramas de Bode.
        - Polos y ceros.
        - Respuestas al impulso, escalón y rampa.
    - **Analizar parámetros temporales**: tiempo de subida, sobreimpulso, pico y establecimiento.
    - **Obtener valores calculados** de los componentes (m, c, k) y (L, R, C).
    """
)


Overwriting 0_📊_Simulacion_Sistemas.py


## Página de Simulación: Sistema Masa–Resorte–Amortiguador

En esta sección definimos la pestaña dedicada al modelo mecánico y su equivalente eléctrico. El usuario podrá:

- **Elegir el tipo de comportamiento** del sistema:  
  - Subamortiguado  
  - Sobreamortiguado  
  - Críticamente amortiguado  
  - Inestable  
- **Ajustar** mediante sliders:  
  - El factor de amortiguamiento \(\zeta\)  
  - La frecuencia natural \(\omega_n\)  
- **Visualizar** en lazo abierto:  
  - Polos y ceros  
  - Parámetros físicos \(m, c, k\) y equivalentes eléctricos \(L, R, C\)  
  - Respuestas al impulso, escalón y rampa  
  - Diagrama de Bode  
- **Obtener** lazo cerrado automáticamente y ver:  
  - Polos del sistema en lazo cerrado  
  - Respuestas temporales (impulso, escalón, rampa)  
  - Parámetros temporales (\(M_p\), \(T_r\), \(T_p\), \(T_s\)) si es subamortiguado  

Cada paso está comentado para explicar su propósito en la generación del dashboard interactivo.


In [ ]:
%%writefile pages/1_🔧_Sistema_Masa_Resorte.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# ── Configuración de la página ────────────────────────────────────────────
st.set_page_config(
    page_title="Sistema Masa-Resorte-Amortiguador",
    layout="wide"
)

# ── Título de la pestaña ──────────────────────────────────────────────────
st.title("🔧 Sistema Masa-Resorte-Amortiguador y Circuito RLC")

# ── Selección del tipo de sistema ────────────────────────────────────────
tipo = st.selectbox(
    "Tipo de sistema",
    ["Subamortiguado", "Sobreamortiguado", "Crítico", "Inestable"]
)

# ── Ajuste de parámetros dinámicos ───────────────────────────────────────
zeta = st.slider("Amortiguamiento ζ", -2.0, 2.5, 0.5, step=0.01)
own = st.slider("Frecuencia natural ωₙ [rad/s]", 0.1, 10.0, 2.0, step=0.1)

# ── Validación del rango de ζ según el tipo de respuesta ────────────────
valid = True
if tipo == "Subamortiguado" and not (0 < zeta < 1):
    st.error("❌ Para subamortiguado, ζ debe estar en (0,1)")
    valid = False
elif tipo == "Sobreamortiguado" and not (zeta > 1):
    st.error("❌ Para sobreamortiguado, ζ debe ser > 1")
    valid = False
elif tipo == "Crítico" and not np.isclose(zeta, 1.0, atol=0.01):
    st.error("❌ Para crítico, ζ ≈ 1")
    valid = False
elif tipo == "Inestable" and not (zeta < 0):
    st.error("❌ Para inestable, ζ debe ser < 0")
    valid = False

if valid:
    # ── Definición de la función de transferencia en lazo abierto ──────────
    num = [own**2]                     # Numerador G(s) = ωₙ²
    den = [1, 2*zeta*own, own**2]      # Denominador s² + 2ζωₙs + ωₙ²
    sistema = signal.TransferFunction(num, den)

    # ── Mostrar polos y ceros del sistema ─────────────────────────────────
    st.subheader("📌 Polos y Ceros (Lazo Abierto)")
    st.write(f"**Polos:** {np.round(sistema.poles, 3)}")
    st.write(f"**Ceros:** {np.round(sistema.zeros, 3)}")

    # ── Cálculo de parámetros físicos y eléctricos equivalentes ────────────
    m = 1.0                            # Masa normalizada
    k = m * own**2                     # Constante del resorte
    c = 2 * zeta * own * m             # Coeficiente de amortiguamiento
    L = m                              # Inductancia equivalente
    R = c                              # Resistencia equivalente
    C = 1 / k                          # Capacitancia equivalente

    st.subheader("⚙️ Parámetros Físicos")
    st.write(f"m = {m:.2f} kg | c = {c:.2f} Ns/m | k = {k:.2f} N/m")
    st.subheader("🔌 Equivalente Eléctrico (R–L–C)")
    st.write(f"L = {L:.2f} H | R = {R:.2f} Ω | C = {C:.3f} F")

    # ── Cálculo de las respuestas temporales ───────────────────────────────
    t = np.linspace(0, 10, 1000)
    _, y_imp = signal.impulse(sistema, T=t)
    _, y_step = signal.step(sistema, T=t)
    _, y_ramp, _ = signal.lsim(sistema, U=t, T=t)

    # ── Generación de gráficos en lazo abierto ─────────────────────────────
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    axs[0, 0].plot(t, y_imp);   axs[0, 0].set_title("Impulso")
    axs[0, 1].plot(t, y_step);  axs[0, 1].set_title("Escalón")
    axs[1, 0].plot(t, y_ramp);  axs[1, 0].set_title("Rampa")
    w, mag, phase = signal.bode(sistema)
    axs[1, 1].semilogx(w, mag, label='Magnitud')
    axs[1, 1].semilogx(w, phase, label='Fase')
    axs[1, 1].set_title("Diagrama de Bode"); axs[1, 1].legend()
    st.pyplot(fig)

    # ── Construcción del sistema en lazo cerrado ───────────────────────────
    sistema_cerrado = signal.TransferFunction(num, np.polyadd(den, num))

    st.subheader("🔄 Sistema en Lazo Cerrado")
    st.write(f"**Polos cerrados:** {np.round(sistema_cerrado.poles, 3)}")

    # ── Respuestas temporales en lazo cerrado ──────────────────────────────
    t_c = t
    _, y_imp_c = signal.impulse(sistema_cerrado, T=t_c)
    _, y_step_c = signal.step(sistema_cerrado, T=t_c)
    _, y_ramp_c, _ = signal.lsim(sistema_cerrado, U=t_c, T=t_c)

    fig2, axs2 = plt.subplots(1, 3, figsize=(18, 4))
    axs2[0].plot(t_c, y_imp_c);  axs2[0].set_title("Impulso (Lazo Cerrado)")
    axs2[0].set_xlabel("Tiempo [s]"); axs2[0].set_ylabel("Amplitud")
    axs2[1].plot(t_c, y_step_c); axs2[1].set_title("Escalón (Lazo Cerrado)")
    axs2[1].set_xlabel("Tiempo [s]")
    axs2[2].plot(t_c, y_ramp_c); axs2[2].set_title("Rampa (Lazo Cerrado)")
    axs2[2].set_xlabel("Tiempo [s]")
    st.pyplot(fig2)

    # ── Cálculo de parámetros temporales para subamortiguado ───────────────
    if 0 < zeta < 1:
        Mp = np.exp(-zeta * np.pi / np.sqrt(1 - zeta**2)) * 100
        Ts = 4 / (zeta * own)
        Tr = (np.pi - np.arccos(zeta)) / (own * np.sqrt(1 - zeta**2))
        Tp = np.pi / (own * np.sqrt(1 - zeta**2))
        st.subheader("⏱️ Parámetros Temporales")
        st.write(f"Mₚ = {Mp:.2f}% | Tᵣ = {Tr:.2f}s | Tₚ = {Tp:.2f}s | Tₛ = {Ts:.2f}s")
    else:
        st.info("ℹ️ Estos parámetros no aplican para el amortiguamiento seleccionado.")


Overwriting pages/1_🔧_Sistema_Masa_Resorte.py


## Ejecución de la Aplicación y Exposición Pública

En este bloque lanzamos la aplicación Streamlit en segundo plano y configuramos un túnel público con Cloudflared para acceder al dashboard desde cualquier lugar. Se realizan los siguientes pasos:

1. **Cierre de instancias previas** de Streamlit y Cloudflared para evitar conflictos.  
2. **Ejecución** del script principal de la aplicación en background.  
3. **Pausa** breve para asegurar que el servidor local esté activo.  
4. **Arranque** del túnel Cloudflared que redirige el puerto local 8501 a una URL accesible públicamente.  
5. **Lectura** del log de Cloudflared para extraer y mostrar la URL pública resultante.


In [ ]:
# 🟢 Ejecuta la app y crea el túnel público

import os
import time
import re

# ── Cerrar procesos antiguos de Streamlit y Cloudflared ────────────────
!pkill -f streamlit                 # Finaliza cualquier instancia previa de Streamlit
!pkill -f cloudflared               # Finaliza cualquier túnel anterior de Cloudflared

# ── Iniciar la aplicación Streamlit en segundo plano ──────────────────
!streamlit run 0_📊_Simulacion_Sistemas.py &> /dev/null &

# ── Esperar unos segundos para que Streamlit esté en línea ────────────
time.sleep(3)

# ── Iniciar el túnel con Cloudflared ──────────────────────────────────
!cloudflared tunnel --url http://localhost:8501 > cloudflared.log 2>&1 &

# ── Dar tiempo para que Cloudflared genere la URL ─────────────────────
time.sleep(8)

# ── Leer el log para extraer la URL pública del túnel ──────────────────
url = None
with open("cloudflared.log") as file:
    for line in file:
        match = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
        if match:
            url = match.group(0)
            break

# ── Mostrar resultado al usuario ───────────────────────────────────────
if url:
    print("✅ Tu dashboard está disponible en:\n", url)
else:
    print("⚠️ No se pudo obtener la URL. Revisa 'cloudflared.log' para más detalles.")


✅ Tu dashboard está disponible en:
 https://closing-wear-movers-convention.trycloudflare.com


## Conclusiones


- **Se comprendió en profundidad** el comportamiento de los sistemas de segundo orden, tanto en su forma mecánica (masa–resorte–amortiguador) como en su equivalente eléctrico (circuito R–L–C).  

- **Se exploró interactivamente** los diferentes regímenes de amortiguamiento (subamortiguado, críticamente amortiguado, sobreamortiguado e inestable) y observar cómo el factor ζ y la frecuencia natural ωₙ afectan la forma de la respuesta temporal y la posición de polos y ceros.  
- **Se visualizó de manera clara** los diagramas de Bode, así como las respuestas al impulso, al escalón y a la rampa, tanto en lazo abierto como en lazo cerrado, reforzando el entendimiento de conceptos clave como la ganancia de pico, el ancho de banda y la estabilidad.  
- **Se calcularon parámetros temporales** —tiempo de subida, sobreimpulso, tiempo al pico y tiempo de establecimiento—, y relacionarlos cuantitativamente con ζ y ωₙ para comprender cómo ajustar un sistema real según requisitos de rendimiento.  
  


